In [ ]:
pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model=keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu', 
        input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter',min_value=32,max_value=128,step=16),
            kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
            activation='relu'
            ),
            keras.layers.Flatten(),
            keras.layers.Dense(
                units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                activation='relu'
            ),
            keras.layers.Dense(10,activation='softmax')#output layer
            ]) 
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])
  return model

  

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist_Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 21m 22s]
val_accuracy: 0.8503333330154419

Best val_accuracy So Far: 0.9089999794960022
Total elapsed time: 01h 33m 08s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |112               
conv_1_kernel     |5                 |3                 
conv_2_filter     |48                |112               
conv_2_kernel     |5                 |5                 
dense_1_units     |96                |96                
learning_rate     |0.001             |0.001             

Epoch 1/3
1688/1688 [==============================] - 496s 293ms/step - loss: 0.4144 - accuracy: 0.8506 - val_loss: 0.3023 - val_accuracy: 0.8858
Epoch 2/3
 666/1688 [==========>...................] - ETA: 4:52 - loss: 0.2727 - accuracy: 0.8984

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()